# GIT CONFIG


In [2]:
pwd

'/content'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/git-colab-repo/movie-genre-classification

/content/drive/MyDrive/git-colab-repo/movie-genre-classification


In [5]:
!git status

Refresh index: 100% (6/6), done.
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   main.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


pastikan dulu kerja di branch apaa

In [6]:
  !git config --global user.email "asmi.amalia23@gmail.com"
  !git config --global user.name "naamalia23"

In [7]:
!git add .
!git commit -m 'git config master'
!git push -u origin master

[master 2761529] git config master
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite main.ipynb (100%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 964 bytes | 321.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/naamalia23/movie-genre-classification
   8de61a7..2761529  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
